### Oppgave 2) Datainnsamling

Dataen hentes ut gjennom en API fra OPEN-METEO.com, Koordinatene som er satt er for Trondheim, og dataen OPEN-METEO bruker for Norge kommer fra datasettet, "MET NORDIC", som er publisert av Norges Meterologiske Institutt. OPEN-METEO er open source, og tar seg ikke betalt for dataen, den er også veldig brukervennlig, og har en liste med alle "keywords" slik at man enkelt kan manøvrere gjennom de ulike dataene. Datasettet lastes ned til en CSV fil og lagres under "data" folderen


In [1]:
import os

# Definer sti til temperaturdata-mappa
data_dir = os.path.abspath(os.path.join(os.getcwd(), '..', 'data', 'temperaturdata'))

# List alle CSV-filer i mappen
datasets = [f for f in os.listdir(data_dir) if f.endswith('.csv')]

print(f"✅ Det er generert {len(datasets)} datasett fra API-et.")
print("Datasett funnet:")
for ds in datasets:
    print(f"  - {ds}")


✅ Det er generert 2 datasett fra API-et.
Datasett funnet:
  - temperaturdata.csv
  - temperaturdata_feilverdier.csv


Datasett nummer 2, " temperaturdata_feilverdier" er generert fra den samme API, med samme keywords. Her er det manuelt plassert feilmålinger og tomme målinger, valideringen av listene i oppgave 3 skal testes på begge datasett slik at man ser at den faktisk finner feil